In [187]:
import os
from azure.ai.inference import ChatCompletionsClient
from azure.ai.inference.models import SystemMessage, UserMessage
from azure.core.credentials import AzureKeyCredential
import json

In [188]:
endpoint = "https://models.inference.ai.azure.com"
model_name = "gpt-4o-mini"
token = os.environ["GITHUB_TOKEN"]

In [189]:
client = ChatCompletionsClient(
    endpoint=endpoint,
    credential=AzureKeyCredential(token),
)

In [190]:
response = client.complete(
    messages=[
        UserMessage(content="What is the capital of France?"),
    ],
    temperature=1.0,
    top_p=1.0,
    max_tokens=1000,
    model=model_name
)
print(response.choices[0].message.content)

The capital of France is Paris.


### Function call simple

In [191]:
from azure.ai.inference.models import (
    AssistantMessage,
    ChatCompletionsToolCall,
    ChatCompletionsToolDefinition,
    CompletionsFinishReason,
    FunctionDefinition,
    SystemMessage,
    ToolMessage,
    UserMessage,
)

In [192]:
# functions=[
#     # Jason file that contian ; name , description paramter
#     #                                              |---type , properties ,required
#     #                                                         |---location ,unit
#     {
#         "name": "get_current_weather",
#         "description": "Get the current weather in a given location",
#         "parameters": {
#             "type": "object",
#             "properties": {
#                 "location": {
#                     "type": "string",
#                     "description": "The city and state, e.g. San Francisco, CA",
#                 },
#                 "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
#             },
#             "required": ["location"],
#         },
#     }
# ]

In [193]:
def get_overall_temp(location, unit="celsius"):
    """Get the current weather in a given location"""
    weather_info = {
        "location": location,
        "temperature": "32",
        "unit": unit,
        "forecast": ["sunny", "windy"],
    }
    return json.dumps(weather_info)

In [194]:
messages = [
    UserMessage(content="I want to know over all temperature of the Thailand"),
]

In [195]:
get_temp = ChatCompletionsToolDefinition(
    function=FunctionDefinition(
        name="get_overall_temp",
        description="""Return Overall temperature of that location """,
        parameters={
            "type": "object",
            "properties": {
                "location": {
                    "type": "string",
                    "description": "The city and state, e.g. San Francisco, CA",
                },
                "Unit": {
                    "type": "string",
                    "enum": ["celsius", "fahrenheit"],
                },
            },
            "required": ["location"],
        },
    )
)

In [196]:
response = client.complete(
    messages=messages,
    model=model_name,
    # temperature=1.0,
    # top_p=1.0,
    # max_tokens=1000,
    tools=[get_temp],
)
print(response.choices[0].message.content)

None


In [197]:
print(response)

{
  "choices": [
    {
      "content_filter_results": {},
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "function": {
              "arguments": "{\"location\":\"Thailand\"}",
              "name": "get_overall_temp"
            },
            "id": "call_knm8JfW88K6fEYwuyrlKIXoZ",
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1737700935,
  "id": "chatcmpl-At7gNtufUwOy1iOG6IP5OgBxGIKEH",
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "prompt_filter_results": [
    {
      "prompt_index": 0,
      "content_filter_results": {
        "hate": {
          "filtered": false,
          "severity": "safe"
        },
        "jailbreak": {
          "filtered": false,
          "detected": false
        },
        "self_harm": {
          "filtered": false

In [198]:
args = json.loads(response.choices[0].message.tool_calls[0].function.arguments)
observation=get_overall_temp(args)
get_overall_temp(args)

'{"location": {"location": "Thailand"}, "temperature": "32", "unit": "celsius", "forecast": ["sunny", "windy"]}'

In [199]:
messages.append(
        {
            "role": "function",
            "name": "get_temp",
            "content": observation,
        }
)

In [200]:
response = client.complete(
    messages=messages,
    model=model_name,
    # temperature=1.0,
    # top_p=1.0,
    # max_tokens=1000,
)
print(response.choices[0].message.content)

The overall temperature in Thailand is around 32°C. The weather is expected to be sunny and windy. If you need more specific information or forecasts for different regions within Thailand, feel free to ask!


### Chain

In [209]:
from langchain.prompts import ChatPromptTemplate
from langchain.chat_models import AzureChatOpenAI #ต้องเป็นAzureChatOpenAI
from langchain.chains import LLMChain
from langchain.schema.output_parser import StrOutputParser
import os

In [210]:
api_key = os.environ['GITHUB_TOKEN']
api_endpoint = "https://models.inference.ai.azure.com"

llm = AzureChatOpenAI(
    openai_api_key=api_key,
    openai_api_base=api_endpoint,
    # deployment_name="your_deployment_name",  # Adjust based on your Azure setup
    model="gpt-4o-mini",  # Specify your model name
    openai_api_version="2023-03-15-preview"  # Ensure it matches your Azure API version
)


/var/folders/cg/kgw71nq93vd05bh5_fkk3_xr0000gn/T/ipykernel_40574/2651002058.py:4: LangChainDeprecationWarning: The class `AzureChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureChatOpenAI``.
  llm = AzureChatOpenAI(
/Library/Frameworks/Python.framework/Versions/3.11/lib/python3.11/site-packages/langchain_community/chat_models/azure_openai.py:174: UserWarning: As of openai>=1.0.0, Azure endpoints should be specified via the `azure_endpoint` param not `openai_api_base` (or alias `base_url`). Updating `openai_api_base` from https://models.inference.ai.azure.com to https://models.inference.ai.azure.com/openai.
  warnings.warn(


In [217]:
prompt = ChatPromptTemplate.from_template(
    "Tell me various short jokes about {topic}"
)

In [218]:
# สร้างChain ออกมา
chain = LLMChain(llm=llm, prompt=prompt)

In [221]:
# Run the chain with a topic
response = chain.run({"topic": "NLP"})
print(response)

Sure! Here are some short jokes about Natural Language Processing (NLP):

1. Why did the NLP model break up with its partner?
   Because they just didn’t understand each other!

2. What did the NLP model say to the unstructured data?
   "You complete me!"

3. Why did the computer go to therapy?
   It had too many unresolved dependencies!

4. How do NLP researchers stay organized?
   They always keep their sentences in order!

5. Why did the NLP model get kicked out of the party?
   It kept trying to tokenize everyone!

6. What do you call a sad NLP model?
   A token of sadness!

7. Why did the word go to school?
   To improve its vocab-ular-ity!

8. How does an NLP model apologize?
   "I’m sorry; that was a parsing error!"

9. Why don't NLP models tell secrets?
   Because they can't keep their embeddings to themselves!

10. What did one vector say to the other?
    "You complete my cosine similarity!"

Hope these brought a smile to your face!


### สามารถเอา Chian นี้ ไปใช้กับระบบ RAG ก่อนที่จะChainเข้าไปใน LLM


In [ ]:
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import DocArrayInMemorySearch